In [ ]:
# display plots inline
%matplotlib notebook

# imports
import os
import numpy as np
import pandas as pd
import pymc3 as pm
from bambi import Model, Prior
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import pymc3_utils as pmu

# suppress system warnings for legibility
import warnings
warnings.filterwarnings('ignore')

# resize plots to fit labels inside bounding box
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

# MPI color scheme
sns.set(style='white', palette='Set2')

# Raven's Progressive Matrices
## Loading data

In [ ]:
# start by loading the data
df_ravens = pd.read_csv('data/ravens.tsv', sep='\t').dropna()
df_reading = pd.read_csv('data/tamil_reading.tsv', sep='\t').drop(columns=['literate', 'subject'])
df_ravens = df_ravens.merge(df_reading, left_on='pp', right_on='pp')

display(df_ravens.head())

## Data mangling and plotting

In [ ]:
# standardize reading score
df_ravens['reading_z'] = pmu.standardize(df_ravens['word'])

display(df_ravens.head())

## Modeling

First we will set some parameters for the regression procedure, as outlined in the paper.

In [ ]:
# Default model params
defaults = {
    'samples': 5000,
    'tune': 2500,
    'chains': 4,
    'init': 'advi+adapt_diag',
    'family': 'bernoulli',
    'priors': {'fixed': 'narrow', 'random': 'narrow'},
}

We are modelling each question in the ravens task as a Bernoulli trial (i.e., a generalized linear model). We will run the only model that works in the context of our multilevel model setup: A model with no fixed effects, only an overall intercept and by-participant and by-item intercepts.

In [ ]:
model_ravens_intercept = Model(df_ravens)
model_ravens_intercept.fit('ACC ~ 1',
                           random=['1|item', '1|pp'],
                           **defaults)

In [ ]:
display(pmu.summary(model_ravens_intercept.backend.trace).round(2))

\\(\hat{r}\\) values look good, as does the number of effective samples for the by-participant intercepts. Even for the overall intercept we have 8.5K effective samples.

## Plot model traces

As an additional check, we will plot the posterior traces for the selected model. Ideally these look unimodal and roughly normal. The plots on the righthand side should look like fuzzy caterpillars.

In [ ]:
g_traces = pm.traceplot(model_ravens_intercept.backend.trace)
plt.savefig('figures/ravens_model_traces.png', dpi=600)

## Extracting participant intercept modes

We will now extract and store the posterior modes of the participant intercepts so we can use them to model reading scores.

In [ ]:
pps = df_ravens['pp'].unique()
pp_nums = [f'1|pp__{i}' for i in range(len(pps))]
df_intercepts = pmu.summary(model_ravens_intercept.backend.trace).loc[pp_nums]
df_intercepts['pp'] = np.sort(pps)

display(df_intercepts.head().round(2))

In [ ]:
df_uncorrected = df_ravens.groupby('pp', as_index=False).mean().rename(columns={'ACC': 'raw_ravens_mean'})
df_intercepts = df_intercepts[['pp', 'mode']].rename(columns={'mode': 'ravens_intercept'})
df_intercepts = df_intercepts.merge(df_uncorrected[['pp', 'reading_z', 'raw_ravens_mean']],
                                    left_on='pp', right_on='pp').reset_index()

display(df_intercepts.head().round(2))

In [ ]:
# and write to file
df_intercepts.to_csv('data/ravens_intercepts.tsv', sep='\t')

Before closing this notebook, we will take a quick look at the correlations between ravens score and reading score.

In [ ]:
display(df_intercepts[['raw_ravens_mean', 'ravens_intercept', 'reading_z']].corr().round(2))